<a href="https://colab.research.google.com/github/bhagirathtallapragada/Secure-AI-project-phase2/blob/main/Federatedlearning2layernet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is an implementation to conduct federated learning using the CIFAR100 dataset with the following settings:
Cases you need to consider:
1) Every party has instances of every class, 2) Every party only holds instances of one class.

Reporting training and test performance in the above settings with 10, 20, 50, 100 parties. Also report how fast the training converges and time it take to train with different number of parties involved.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow_federated==0.13.1 --quiet

     |████████████████████████████████| 428 kB 12.7 MB/s 
     |████████████████████████████████| 104 kB 49.0 MB/s 
     |████████████████████████████████| 82 kB 1.0 MB/s 
     |████████████████████████████████| 93 kB 2.4 MB/s 
     |████████████████████████████████| 2.2 MB 44.4 MB/s 
     |████████████████████████████████| 1.0 MB 76.5 MB/s 
     |████████████████████████████████| 20.0 MB 473 kB/s 
     |████████████████████████████████| 422.0 MB 33 kB/s 
     |████████████████████████████████| 2.8 MB 53.2 MB/s 
     |████████████████████████████████| 3.8 MB 51.4 MB/s 
     |████████████████████████████████| 448 kB 67.6 MB/s 
     |████████████████████████████████| 41 kB 956 kB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you have gast 0.2.2 whi

In [ ]:
# !pip install --upgrade tensorflow --quiet
# !pip install tensorflow

In [ ]:
# !pip uninstall --yes tensorboard tb-nightly
# !pip install --quiet --upgrade tensorflow-federated-nightly
# !pip install --quiet --upgrade nest-asyncio
# !pip install --quiet --upgrade tb-nightly

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
%load_ext tensorboard

In [ ]:
import collections
import time
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tensorflow.keras import utils as np_utils
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
# from keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import CategoricalCrossentropy
np.random.seed(0)
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   
torch.backends.cudnn.benchmark=True
from torch.utils.data import DataLoader, Dataset

# Next version

In [ ]:
(x_train_100, y_train_100), (x_test_100, y_test_100) = cifar100.load_data()
x_train_100 = x_train_100.astype('float32')
x_test_100 = x_test_100.astype('float32')
x_train_100  /= 255
x_test_100 /= 255

169009152/169001437 [==============================] - 6s 0us/step


In [ ]:
def ext_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [ ]:
#extract equal ratio of all classes for training data
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# for the test case
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
np.unique(y_test, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 1

## 1) Every party only holds instances of one class

#### 1) 10 Clients

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:
# parameters
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
    for  i in range(98):
        model.add( tf.keras.layers.Dense( 64, activation='relu' ) )
    model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

    return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 0 <sparse_categorical_accuracy=0.9959999918937683,loss=0.03382914885878563,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9964600205421448,loss=0.02134837582707405,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9963799715042114,loss=0.015984635800123215,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.9965199828147888,loss=0.

### Evaluation

In [ ]:
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.13699999451637268,loss=2.543506145477295,keras_training_time_client_sum_sec=0.0>


### 2) 20 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
# parameters
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9960399866104126,loss=0.03697780519723892,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9962000250816345,loss=0.026739446446299553,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9962400197982788,loss=0.02325989305973053,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.996209979057312,loss=0.02068919688463211,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.9962499737739563,loss=0.018624814227223396,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.9962400197982788,loss=0.017137035727500916,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.9962800145149231,loss=0.015796171501278877,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.9962999820709229,loss=0.01506357453763485,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_a

### Evaluation

In [ ]:
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.0989999994635582,loss=3.0804636478424072,keras_training_time_client_sum_sec=0.0>


### 3) 50 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [ ]:
# parameters
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9959440231323242,loss=0.037718288600444794,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9960839748382568,loss=0.031111428514122963,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9960759878158569,loss=0.028964880853891373,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.9960799813270569,loss=0.02710742875933647,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.9960960149765015,loss=0.025624044239521027,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.9961040019989014,loss=0.02401348017156124,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.9961000084877014,loss=0.022827167063951492,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.9960960149765015,loss=0.021889837458729744,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorica

### Evaluation

In [ ]:
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.026200000196695328,loss=4.331847190856934,keras_training_time_client_sum_sec=0.0>


### 3) 100 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [ ]:
# parameters
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9960399866104126,loss=0.03543030098080635,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9960319995880127,loss=0.03061618283390999,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9960259795188904,loss=0.028964536264538765,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.9960280060768127,loss=0.027742359787225723,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.9960299730300903,loss=0.02672729082405567,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.9960280060768127,loss=0.025807589292526245,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.9960359930992126,loss=0.02511356584727764,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.9960399866104126,loss=0.024394679814577103,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_

### Evaluation

In [ ]:
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.010200000368058681,loss=4.990439414978027,keras_training_time_client_sum_sec=0.0>


In [ ]:
state, metrics = algorithm.next( state, data )

In [ ]:
metrics

AnonymousTuple([('sparse_categorical_accuracy', 0.99), ('loss', 0.054717205), ('keras_training_time_client_sum_sec', 0.0)])

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.011900000274181366,loss=4.932197093963623,keras_training_time_client_sum_sec=0.0>


## 1) Every party has instances of every class

In [ ]:
def ext_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = ext_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [ ]:
x_new = []
y_new = []
for i in range(500):
    for j in range(x_train.shape[0]//500):
        x_new.append(x_train[j*500+i])
        y_new.append(y_train[j*500+i])

In [ ]:
x_new = np.array(x_new)
x_new.shape

(50000, 3072)

In [ ]:
y_new = np.array(y_new)
y_new.shape

(50000,)

### 1) 10 Clients

In [ ]:
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:

# parameters
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 30

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.05278199911117554,loss=4.266721725463867,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.11944799870252609,loss=3.8055810928344727,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.1584240049123764,loss=3.5800442695617676,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.18024800717830658,loss=3.458298683166504,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.1968819946050644,loss=3.3686110973358154,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.2101839929819107,loss=3.2939107418060303,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.22045199573040009,loss=3.233283519744873,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.23027199506759644,loss=3.178624391555786,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0.

### Evaluation

In [ ]:
NUM_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.2160000056028366,loss=3.3676819801330566,keras_training_time_client_sum_sec=0.0>


### 2) 20 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# parameters
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 30

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.03710800036787987,loss=4.415712356567383,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.09107399731874466,loss=3.982799768447876,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.13134799897670746,loss=3.7303619384765625,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.1572359949350357,loss=3.595865488052368,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.17455799877643585,loss=3.5044467449188232,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.18874800205230713,loss=3.4305524826049805,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.19981999695301056,loss=3.366664171218872,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.2102780044078827,loss=3.3098437786102295,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0

### Evaluation

In [ ]:
NUM_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.19679999351501465,loss=3.478153944015503,keras_training_time_client_sum_sec=0.0>


### 3) 50 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# parameters
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 30

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.0303179994225502,loss=4.498284339904785,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.06439799815416336,loss=4.212356090545654,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.09606199711561203,loss=3.950756549835205,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.11937999725341797,loss=3.8029637336730957,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.13729199767112732,loss=3.692713499069214,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.15332800149917603,loss=3.6051816940307617,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.16589200496673584,loss=3.536808967590332,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.17592200636863708,loss=3.478910207748413,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0.

### Evaluation

In [ ]:
NUM_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.16290000081062317,loss=3.6425647735595703,keras_training_time_client_sum_sec=0.0>


### 4) 100 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = ext_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
# parameters
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
BATCH_SIZE = 20
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ

In [ ]:
# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )

In [ ]:
# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.024922000244259834,loss=4.54674768447876,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.04081999883055687,loss=4.447535514831543,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.061785999685525894,loss=4.275726795196533,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.0836699977517128,loss=4.071457862854004,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.10298199951648712,loss=3.918820381164551,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.11763600260019302,loss=3.8149516582489014,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.13095399737358093,loss=3.7343668937683105,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.1424880027770996,loss=3.667412757873535,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0.

### Evaluation

In [ ]:
NUM_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NUM_CLIENT_SAMPLES = TOTAL_SAMPLES // NUM_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NUM_CLIENTS ):
  x = x_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES  ].reshape(NUM_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NUM_CLIENT_SAMPLES : ( i + 1 ) * NUM_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.1396999955177307,loss=3.7924530506134033,keras_training_time_client_sum_sec=0.0>
